In [1]:
import json
import salem
import math
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
with open("../1_Country_Emission/Countries.json",'r') as load_f:
    Countries = json.load(load_f)

In [3]:
NCDLRI = pd.read_csv('C:/Users/15311/Desktop/InMAP/Mortality Number/NCD_LRI.csv')
code = pd.read_csv('C:/Users/15311/Desktop/InMAP/Mortality Number/code.csv')

In [4]:
NCDLRI = NCDLRI[['year','location_name','sex_name','age_name','cause_name','metric_name','val']]
NCDLRI = NCDLRI[NCDLRI['metric_name']  == 'Rate']

In [5]:
LRI = NCDLRI[NCDLRI['cause_name'] == 'Lower respiratory infections'].reset_index(drop=True)
NCD = NCDLRI[NCDLRI['cause_name'] == 'Non-communicable diseases'].reset_index(drop=True)

In [6]:
code.columns = ['location_name','code']

In [7]:
left_countries = []
for i in NCD['location_name']:
    if i not in list(code['location_name']):
        left_countries.append(i)

There are some countries in GBD dataset but not in countries' codes dataset.

In [8]:
left_countries

['North Macedonia',
 'Palestine',
 'Bolivia (Plurinational State of)',
 'Cabo Verde',
 'Eswatini',
 'Libya',
 'South Sudan',
 'Micronesia (Federated States of)',
 'Iran (Islamic Republic of)',
 'Republic of Moldova']

In [9]:
NCD_left = 0
LRI_left = 0
for i in left_countries:
    NCD_left += NCD[NCD['location_name'] == i]['val'].values[0]
    LRI_left += LRI[LRI['location_name'] == i]['val'].values[0]

In [10]:
NCD_left,LRI_left # Add those number of death to "ROW"

(5328.531526594077, 357.3165244565098)

In [11]:
LRI = pd.merge(LRI,code)
NCD = pd.merge(NCD,code)

In [12]:
NCD

,year,location_name,sex_name,age_name,cause_name,metric_name,val,code
0,2019,Sri Lanka,Both,All Ages,Non-communicable diseases,Rate,511.975970,LKA
1,2019,Dominica,Both,All Ages,Non-communicable diseases,Rate,924.425096,DMA
2,2019,Viet Nam,Both,All Ages,Non-communicable diseases,Rate,526.878035,VNM
3,2019,Germany,Both,All Ages,Non-communicable diseases,Rate,1043.915933,DEU
4,2019,Kiribati,Both,All Ages,Non-communicable diseases,Rate,693.395406,KIR
...,...,...,...,...,...,...,...,...
189,2019,Burkina Faso,Both,All Ages,Non-communicable diseases,Rate,294.262605,BFA
190,2019,Nauru,Both,All Ages,Non-communicable diseases,Rate,452.025431,NRU
191,2019,Sudan,Both,All Ages,Non-communicable diseases,Rate,319.602616,SDN
192,2019,Chad,Both,All Ages,Non-communicable diseases,Rate,241.486625,TCD


In [13]:
missing_countries = []
for i in Countries:
    if i in NCD['code'].unique():
        continue    
    else:
        missing_countries.append(i)

In [14]:
missing_countries

['ABW',
 'BOL',
 'VGB',
 'CPV',
 'CYM',
 'PYF',
 'HKG',
 'IRN',
 'LBY',
 'LIE',
 'MAC',
 'ANT',
 'NCL',
 'PSE',
 'MDA',
 'SDS',
 'SUD',
 'SWZ',
 'MKD',
 'USR',
 'ROW']

# choose a region has similar GPD and in the same region.
# or: Assign those countries according to world bank region

In [15]:
df_shp = salem.read_shapefile(r'C:/Users/15311/Desktop/natural_earth_vector/10m_cultural/ne_10m_admin_0_countries.shp')

In [16]:
df_shp[df_shp['ADM0_A3'].isin(['PSX','SDN'])]

,featurecla,scalerank,LABELRANK,SOVEREIGNT,SOV_A3,ADM0_DIF,LEVEL,TYPE,ADMIN,ADM0_A3,...,NAME_RU,NAME_SV,NAME_TR,NAME_VI,NAME_ZH,geometry,min_x,max_x,min_y,max_y
11,Admin-0 country,0,5,Israel,IS1,1,2,Disputed,Palestine,PSX,...,Палестина,Palestina,Filistin,Palestine,巴勒斯坦,"MULTIPOLYGON (((34.48120 31.58314, 34.51141 31...",34.200269,35.572536,31.211449,32.542640
86,Admin-0 country,0,3,Sudan,SDN,0,2,Sovereign country,Sudan,SDN,...,Судан,Sudan,Sudan,Sudan,苏丹共和国,"MULTIPOLYGON (((22.86106 10.91915, 22.90023 11...",21.809449,38.603852,8.681642,22.226965


In [17]:
df_shp.loc[11,'ADM0_A3'], df_shp.loc[86,'ADM0_A3']

('PSX', 'SDN')

### Change 'PSX' to 'PSE' and change 'SDN' to  'SUD'

In [18]:
df_shp.loc[11,'ADM0_A3'] = 'PSE'
df_shp.loc[86,'ADM0_A3'] = 'SUD'

In [19]:
df_shp.loc[11,'ADM0_A3'], df_shp.loc[86,'ADM0_A3']

('PSE', 'SUD')

In [20]:
subset = df_shp[df_shp["ADM0_A3"].isin(missing_countries[:-1])]

In [21]:
# subset.columns # ‘REGION_WB’

In [22]:
for i in missing_countries:
    if i in subset['ADM0_A3'].unique():
        continue
    else:
        print(i)

ANT
USR
ROW


In [23]:
# np.sort(list(NCD['location_name'].unique()))

In [24]:
for i in range(len(LRI)):
    if LRI.iloc[i,7] not in Countries:
        LRI.iloc[i,7] = 'ROW' ## No.7 is the country code
        
for i in range(len(NCD)):
    if NCD.iloc[i,7] not in Countries:
        NCD.iloc[i,7] = 'ROW'

In [25]:
NCD_dict = {}
LRI_dict = {}
for i in Countries:
    NCD_dict[i] = 0
    LRI_dict[i] = 0

In [26]:
GBD_WB = pd.read_csv('C:/Users/15311/Desktop/InMAP/Mortality Number/GBD_WB.csv')

In [27]:
GBD_LRI = GBD_WB[GBD_WB['cause_name'] == 'Lower respiratory infections']
GBD_NCD = GBD_WB[GBD_WB['cause_name'] == 'Non-communicable diseases']

In [28]:
GBD_NCD

,measure_id,measure_name,location_id,location_name,sex_id,sex_name,age_id,age_name,cause_id,cause_name,metric_id,metric_name,year,val,upper,lower
1,1,Deaths,242,Sub-Saharan Africa,3,Both,22,All ages,409,Non-communicable diseases,3,Rate,2019,255.335794,284.967473,228.769836
2,1,Deaths,243,Middle East & North Africa,3,Both,22,All ages,409,Non-communicable diseases,3,Rate,2019,390.945354,431.078600,353.639951
5,1,Deaths,240,Europe & Central Asia,3,Both,22,All ages,409,Non-communicable diseases,3,Rate,2019,917.268621,950.181120,885.399857
7,1,Deaths,244,South Asia,3,Both,22,All ages,409,Non-communicable diseases,3,Rate,2019,425.952201,472.436628,381.652761
8,1,Deaths,44638,North America,3,Both,22,All ages,409,Non-communicable diseases,3,Rate,2019,794.869312,802.461319,788.028926
10,1,Deaths,239,East Asia & Pacific,3,Both,22,All ages,409,Non-communicable diseases,3,Rate,2019,633.757233,688.990669,580.198177
12,1,Deaths,241,Latin America & Caribbean,3,Both,22,All ages,409,Non-communicable diseases,3,Rate,2019,483.238895,518.297211,451.781575


In [29]:
NCD_dict = {}
LRI_dict = {}
count = 0
for i in Countries[:-1]:
    if i in list(subset[subset['REGION_WB'] == 'Latin America & Caribbean']['ADM0_A3']):
        NCD_dict[i] = GBD_NCD[GBD_NCD['location_name'] == 'Latin America & Caribbean']['val'].values[0]
        LRI_dict[i] = GBD_LRI[GBD_LRI['location_name'] == 'Latin America & Caribbean']['val'].values[0]

    if i in list(subset[subset['REGION_WB'] == 'Middle East & North Africa']['ADM0_A3']):
        NCD_dict[i] = GBD_NCD[GBD_NCD['location_name'] == 'Middle East & North Africa']['val'].values[0]
        LRI_dict[i] = GBD_LRI[GBD_LRI['location_name'] == 'Middle East & North Africa']['val'].values[0]
        
    if i in list(subset[subset['REGION_WB'] == 'Sub-Saharan Africa']['ADM0_A3']):
        NCD_dict[i] = GBD_NCD[GBD_NCD['location_name'] == 'Sub-Saharan Africa']['val'].values[0]
        LRI_dict[i] = GBD_LRI[GBD_LRI['location_name'] == 'Sub-Saharan Africa']['val'].values[0]
        
    if i in list(subset[subset['REGION_WB'] == 'Europe & Central Asia']['ADM0_A3']):
        NCD_dict[i] = GBD_NCD[GBD_NCD['location_name'] == 'Europe & Central Asia']['val'].values[0]
        LRI_dict[i] = GBD_LRI[GBD_LRI['location_name'] == 'Europe & Central Asia']['val'].values[0]
        
    if i in list(subset[subset['REGION_WB'] == 'East Asia & Pacific']['ADM0_A3']):
        NCD_dict[i] = GBD_NCD[GBD_NCD['location_name'] == 'East Asia & Pacific']['val'].values[0]
        LRI_dict[i] = GBD_LRI[GBD_LRI['location_name'] == 'East Asia & Pacific']['val'].values[0]
        
    if i not in missing_countries:
        NCD_dict[i] = NCD[NCD['code'] == i]['val'].values[0]
        LRI_dict[i] = LRI[LRI['code'] == i]['val'].values[0]

NCD_dict['ROW'] = sum(NCD[NCD['code'] == 'ROW']['val']) + NCD_left
LRI_dict['ROW'] = sum(LRI[LRI['code'] == 'ROW']['val']) + LRI_left

In [30]:
subset[['SOVEREIGNT','ADM0_A3','REGION_WB']]

,SOVEREIGNT,ADM0_A3,REGION_WB
3,Bolivia,BOL,Latin America & Caribbean
11,Israel,PSE,Middle East & North Africa
14,South Sudan,SDS,Sub-Saharan Africa
62,Macedonia,MKD,Europe & Central Asia
72,Libya,LBY,Middle East & North Africa
86,Sudan,SUD,Sub-Saharan Africa
93,Iran,IRN,Middle East & North Africa
95,Liechtenstein,LIE,Europe & Central Asia
123,eSwatini,SWZ,Sub-Saharan Africa
140,Moldova,MDA,Europe & Central Asia


In [31]:
NCDLRI_Total = {}
for i in NCD_dict:
    NCDLRI_Total[i] = LRI_dict[i] + NCD_dict[i]

In [32]:
len(NCDLRI_Total),len(LRI_dict),len(NCD_dict) # Without ANT and USR

(188, 188, 188)

In [33]:
NCDLRI_Total

{'AFG': 393.29797883455456,
 'ALB': 792.5125714952795,
 'DZA': 406.4376197665684,
 'AND': 688.4260139076165,
 'AGO': 265.4001210610221,
 'ATG': 628.6801700088407,
 'ARG': 700.5734129318575,
 'ARM': 878.8438262737986,
 'ABW': 519.1491889399999,
 'AUS': 649.6549614713771,
 'AUT': 871.5601372592926,
 'AZE': 670.7095794838159,
 'BHS': 588.3670973762185,
 'BHR': 262.72470932835404,
 'BGD': 420.8112032072144,
 'BRB': 967.5407585952158,
 'BLR': 1184.4683181926232,
 'BEL': 915.6098817813166,
 'BLZ': 360.79238628789284,
 'BEN': 320.46265732612164,
 'BMU': 855.5935169494595,
 'BTN': 431.88746245087276,
 'BOL': 519.1491889399999,
 'BIH': 1089.5077105305406,
 'BWA': 458.76895384497186,
 'BRA': 535.3041143654063,
 'VGB': 519.1491889399999,
 'BRN': 378.547013163599,
 'BGR': 1733.5855540830985,
 'BFA': 401.7538715834496,
 'BDI': 293.2120373048849,
 'KHM': 515.2145698745526,
 'CMR': 309.762055482363,
 'CAN': 731.3388806824655,
 'CPV': 316.47413911,
 'CYM': 519.1491889399999,
 'CAF': 443.6538195749584,

In [34]:
NCDLRI_Total['CHN']

687.7392415650974

In [35]:
NCDLRI_Total['HKG'],NCDLRI_Total['MAC']

(658.00550288, 658.00550288)

In [38]:
# Let Hong Kong and Macao have the same mortality rate as mainland China
NCDLRI_Total['HKG'],NCDLRI_Total['MAC'] = NCDLRI_Total['CHN'], NCDLRI_Total['CHN']

In [39]:
NCDLRI_Total['HKG'],NCDLRI_Total['MAC']

(687.7392415650974, 687.7392415650974)

In [41]:
sorted(NCDLRI_Total.items(), key=lambda x: x[1], reverse=True)

[('ROW', 22826.022132241025),
 ('BGR', 1733.5855540830985),
 ('UKR', 1455.7173694662133),
 ('LVA', 1341.9450142837588),
 ('MCO', 1329.5255666984694),
 ('ROU', 1305.339981074029),
 ('SRB', 1293.2636253027545),
 ('LTU', 1277.5164280121287),
 ('GEO', 1275.0355312813795),
 ('HUN', 1263.4659832415007),
 ('GRC', 1202.2222381948066),
 ('BLR', 1184.4683181926232),
 ('HRV', 1161.9625854787798),
 ('EST', 1150.2075927345663),
 ('RUS', 1099.5566445967816),
 ('BIH', 1089.5077105305406),
 ('DEU', 1073.4702584817726),
 ('MNE', 1040.787938737988),
 ('PRT', 1033.7816464258503),
 ('JPN', 1029.8400926372115),
 ('ITA', 1014.6194606981603),
 ('CZE', 1010.3958160576224),
 ('POL', 997.0327114698315),
 ('BRB', 967.5407585952158),
 ('FIN', 952.6958231456798),
 ('SVK', 948.0919983996175),
 ('LIE', 946.7455651599998),
 ('MDA', 946.7455651599998),
 ('MKD', 946.7455651599998),
 ('SVN', 933.0076672608096),
 ('BEL', 915.6098817813166),
 ('DNK', 901.658304591316),
 ('URY', 892.10028546346),
 ('ESP', 887.5286790015989

In [40]:
file=open("NCDLRI_Total.json",'w')
json.dump(NCDLRI_Total,file)
file.close()